In [225]:
import pandas as pd
import json
import numpy as np
import os
import glob
import re
import urllib
from bs4 import BeautifulSoup
from pathlib import Path


In [226]:
def ig_unknown(file, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [227]:
def ig_isotope(measured_df, mat_list, hl_min, hl_max, name):
    precursores = mat_list.split(',')
    isotopes = []
    with open("database/irr_products.json") as f:
        irradiation_products = json.load(f)
    
    for precursor in precursores:
        try:
            isotopes += irradiation_products[precursor]
        
        except:
            irradiation_products[precursor] = load_irr_prod(precursor)
            with open("database/irr_products.json", 'w') as json_out:
                json.dump(irradiation_products, json_out)

    # get unique only
    isotopes = list(set(isotopes))

    db_parameters = ["half-life", "sigm_half-life", "lambda", "sigm_lambda", "isotope", "Ig", "sigm_Ig", "E_tab", "sigm_E_tab"]

    measured_df[[db_parameters]] = np.nan*np.ones(shape=(measured_df.shape[0], len(db_parameters)))

    for isotope in isotopes:
        measured_df = add_db_info(measured_df, isotope)
    
    measured_df.to_csv(f"out/{name}wDBinfo.csv")

In [228]:
def ig_nat_element(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [229]:
def ig_element_spec(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [230]:
def ig_material(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [231]:
#def get_Igamma(file, material, hl_min, hl_max):
#    df = pd.read_csv(file,header=0,index_col=0)
#    name = file.split('.')[0]
#    mat_type, mat_list = material.split(',', maxindex=1)
#    switcher = {
#        0: ig_unknown(df, hl_min, hl_max, name),
#        "isotope": ig_isotope(df, mat_list, hl_min, hl_max, name), 
#        "element_natural": ig_nat_element(df, mat_list, hl_min, hl_max, name),
#        "element_spec": ig_spec_element(df, mat_list, hl_min, hl_max, name),
#        "material": ig_material(df, mat_list, hl_min, hl_max, name)
#    }
#    switcher.get(mat_type, "Invalid material given")

In [232]:
def append_Igamma_dir(parsed_dir):
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")

    #Path("./reports_Ig").mkdir(parents=True, exist_ok=True)
    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        append_Igamma(parsed_file)

In [233]:
def append_Igamma(file):
    print(file)
    A, element = re.split(r'(\d+)(\w+)', file)[-3:-1]
    print(A)
    print(element)

    Path("./ig_db").mkdir(parents=True, exist_ok=True)
    if os.path.isfile(f"ig_db/{A}{element}.csv"):
        ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
        print("Reading the csv with gammas.")
    else:
        Path("./downloads").mkdir(parents=True, exist_ok=True)
        if os.path.isfile(f"downloads/{A}{element}.html"):
            extract_Igamma(A, element)
            ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
            print("Reading the csv with gammas.")
        else:
            download(A, element)
            extract_Igamma(A, element)
            ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
            print("Reading the csv with gammas.")
    
    parsed_df = pd.read_csv(file, index_col=0)
    joined_df = add_Ig(parsed_df, ig_df)
    
    Path("./with_Ig").mkdir(parents=True, exist_ok=True)
    joined_df.to_csv(f"with_Ig/{A}{element}.csv")


In [234]:
def add_Ig(df, ig):
    e_col = []
    ig_col = []

    a = np.empty((df.shape[0], 2))
    a[:] = np.nan
    df[["E_tab", "Ig"]] = a
    # UGLY, never for-loop in pandas
    for row_no in range(df.shape[0]):
        for ig_row_no in range(ig.shape[0]):
            if (df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E_tab"].iloc[ig_row_no]:
                continue
            elif (df["Energy"] + df["FWHM"]).iloc[row_no] < ig["E_tab"].iloc[ig_row_no]:
                break
            else:
                df.loc[row_no, ["E_tab", "Ig"]] = ig.loc[ig_row_no, ["E_tab", "Ig"]]
    print("Ig added")
    return df

In [235]:
def getZ(element):
    element_Z ={
        "Eu": 63,
        "U": 92,
        "Pu": 94
    }
    return element_Z[element]

In [236]:
def download(A, element):
    url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={getZ(element)}0{A}"
    urllib.request.urlretrieve (url, f"downloads/{A}{element}.html")
    print(f"File {A}{element}.html downloaded.")

In [237]:
def extract_Igamma(A, element):
    html_file = open(f"downloads/{A}{element}.html", "r")
    soup = BeautifulSoup(html_file.read(), 'lxml')
    
    gammas_table = soup.find_all("table")[4]
    gammas_rows = gammas_table.find_all('tr')[3:-1]
    energy = []
    sigm_energy = []
    i = []
    sigm_i = []

    for row in gammas_rows:
        cells = row.find_all('td')
        
        e_val = cells[0].get_text(strip=True)
        i_val = cells[1].get_text(strip=True)
        try:
            ig_val = float(i_val[:-1])
            sigm_ig_val = float(i_val[-1])
        except:
            ig_val = float('NaN')
            sigm_ig_val = float('NaN')
       
        energy.append(float(e_val[:-1]))
        sigm_energy.append(int(e_val[-1]))
        i.append(ig_val)
        
        sigm_i.append(sigm_ig_val)


    df_dict = {
        "E_tab": energy,
        "sigm_E_tab": sigm_energy, 
        "Ig": i,
        "sigm_Ig": sigm_i
        }
    df = pd.DataFrame(df_dict)
    df_name = f'ig_db/{A}{element}.csv'
    df.to_csv(df_name)
   
    print(f"Ig extracted from file 'downloads/{A}{element}.html' into '{df_name}'.")

In [238]:
append_Igamma_dir("parsed_reports")

parsed_reports/152Eu.csv
152
Eu
Reading the csv with gammas.
Ig added
